In [89]:
import cudf
import numpy as np

In [2]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

' \n+++++ UTILITY ATTRIBUTES +++++\n    \n    user_code --> String --> (Anonymized) code for the customer that owns this utility\n    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized\n    city --> String --> City where the utility is located\n    address --> String --> (Anonymized) address of the utility location\n'

In [3]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

' \n+++++ CUSTOMER ATTRIBUTES +++++\n    \n    user_code --> String --> (Anonymized) code that identifies the customer\n    nominative --> String --> (Anonymized) customer name\n    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)\n    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18\n'

In [4]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

' \n+++++ INVOICE ATTRIBUTES +++++\n    \n    bill_id --> Int --> Invoice identifier\n    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot\n    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot\n    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot \n    date --> Date --> Start date\n    light_start_date --> Date --> Start date of electricity invoice\n    light_end_date --> Date --> End date of electricity invoice\n    tv --> Float --> Television fee to pay\n    gas_amount --> Float --> Gas fee to pay\n    gas_average_cost --> Float --> Average cost of gas\n    light_average_cost --> Float --> Average cost of electricity\n    emission_date --> Date --> Emission date\n    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)\n    gas_start_date --> Date --> Start date of gas invoice\n    gas_end_date --> Date --> End date of gas invoice\n    extra_fees --> Float --> Extra fees to pay\n    gas_consumption --> Fl

In [5]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli
    • Locate missing values
    • Locate outliers
    • Sort data
'''

' \n+++++ DATA INGESTION +++++\n    • Read data from its source\n    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli\n    • Locate missing values\n    • Locate outliers\n    • Sort data\n'

In [90]:
from base import *
base = BaseDfBench()

In [91]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

base.load_dataset('/data/invoices_sample.csv', 'csv')
df = base.get_df()
columns = base.get_columns()
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,26.0,25.0,33.0,CINTANO,2D244E5D90065985546AEA4D52F54C9705B8038B,102CADC9FC8C745FE3B8C27F7267E07C9691728A,M,55,6D3FDD0A2EEEE1A2C1AC23E6C316048DD2CCE859,...,<NA>,53D29ACDEB11F45D4F305A4D00E5F03C1F2CB674,monthly,False,<NA>,13.08,<NA>,7.62,<NA>,15.65
1996,1996,71.0,61.0,88.0,TREZZONE,A01243282C24F561D6B54F1C1FCB45AE80F75B0A,9C3B5B4093FC99569E5BBA918A977681A46F52DF,M,48,52315237533E1A6504E3795EAC1D5586744C62FD,...,<NA>,2B8D02A2AD117EC401BCEF3C0651FEEBB6609D1C,monthly,False,<NA>,9.2,<NA>,8.76,<NA>,25.8
1997,1997,37.0,71.0,100.0,RIPATRANSONE,469AB252FF0A2C60B32D0337C933E3565E88C0C2,1B451B6AA312EFF93372D44EC6C24C0110A628A1,F,32,2777855F19667FBFEDEAEF181E7ABC8584262127,...,<NA>,A1BFA0E4833E27AB398F4420414E037D250F0A7E,monthly,False,<NA>,8.7,<NA>,8.66,<NA>,24.14
1998,1998,74.0,78.0,92.0,MONSELICE,9E24BB26322BD95D11E26DFB80AA1168C9661FDC,F6BADD2456E283A8DCA6F3FF640724072108BAEF,F,54,8DC8820FB50005ED9CC4E7C187B9703460941856,...,<NA>,DED078288677E4E8A15874405E79C26AC78747B0,monthly,False,<NA>,10.21,<NA>,8.96,<NA>,30.43


In [8]:
#verica di user_code+customer_code = PK utility
df.groupby(by=['user_code', 'customer_code']).count()

#PK customer = user_code

#PK ivoice = bill_id


bill_id  \
user_code                                customer_code                                       
00106F4E6FEEFB8C81BADE0E7E985D85DFA3B9D9 B65DF5CEC830DF4EA6D402004F67F8597DC277DB        1   
                                         D1C6F205FC9B53080CD029948223269294E37770        1   
0045FE85440E191D5FEE535579C0FAA07A92D051 C82AB2965E72B4AF88C31C0D49ADE62C3164D714        1   
005FA7FE8EAA363BE15C61DEE250DA4CA078B462 E71684D5336E0AB1489BACE025474945B1BA288A        1   
006BD4771607483730EE2301634ACFC5B2B57C04 28478BBFFC0BF9F67CDB68090AC1AAA37944163F        1   
...                                                                                    ...   
FFBCAFB0F9ABF49EE7CFE9169E1F6C09AE55711B A0BF38F1974BE8369ECE4D2C97F4F7600F13817A        1   
FFCCFA289099A7FF44726330B8B2C8730411C950 D287E2D6BA2F8656D0869E03D988CEBF7B836D61        1   
FFD9607191459984619642FFDE109415FBD135D4 150A285643FD1A1F75B3DAB3B909C60402F03D76        1   
FFDFE2EDE8A46EF89F4E670028002763FC719E22 32BAC780CED052F95E2A8A8536F48264A1C08756        1   
FFE754BB2D984795616DAF79F744A209E4770B12 28252AC948BB1B4419C2CB07D6DDBAF763F341BF        1   

                                                                                   F1_kWh  \
user_code                                customer_code                                      
00106F4E6FEEFB8C81BADE0E7E985D85DFA3B9D9 B65DF5CEC830DF4EA6D402004F67F8597DC277DB       1   
                                         D1C6F205FC9B53080CD029948223269294E37770       1   
0045FE85440E191D5FEE535579C0FAA07A92D051 C82AB2965E72B4AF88C31C0D49ADE62C3164D714       1   
005FA7FE8EAA363BE15C61DEE250DA4CA078B462 E71684D5336E0AB1489BACE025474945B1BA288A       1   
006BD4771607483730EE2301634ACFC5B2B57C04 28478BBFFC0BF9F67CDB68090AC1AAA37944163F       1   
...                                                                                   ...   
FFBCAFB0F9ABF49EE7CFE9169E1F6C09AE55711B A0BF38F1974BE8369ECE4D2C97F4F7600F13817A       1   
FFCCFA289099A7FF44726330B8B2C8730411C950 D287E2D6BA2F8656D0869E03D988CEBF7B836D61       1   
FFD9607191459984619642FFDE109415FBD135D4 150A285643FD1A1F75B3DAB3B909C60402F03D76       1   
FFDFE2EDE8A46EF89F4E670028002763FC719E22 32BAC780CED052F95E2A8A8536F48264A1C08756       1   
FFE754BB2D984795616DAF79F744A209E4770B12 28252AC948BB1B4419C2CB07D6DDBAF763F341BF       1   

                                                                                   F2_kWh  \
user_code                                customer_code                                      
00106F4E6FEEFB8C81BADE0E7E985D85DFA3B9D9 B65DF5CEC830DF4EA6D402004F67F8597DC277DB       1   
                                         D1C6F205FC9B53080CD029948223269294E37770       1   
0045FE85440E191D5FEE535579C0FAA07A92D051 C82AB2965E72B4AF88C31C0D49ADE62C3164D714       1   
005FA7FE8EAA363BE15C61DEE250DA4CA078B462 E71684D5336E0AB1489BACE025474945B1BA288A       1   
006BD4771607483730EE2301634ACFC5B2B57C04 28478BBFFC0BF9F67CDB68090AC1AAA37944163F       1   
...                                                                                   ...   
FFBCAFB0F9ABF49EE7CFE9169E1F6C09AE55711B A0BF38F1974BE8369ECE4D2C97F4F7600F13817A       1   
FFCCFA289099A7FF44726330B8B2C8730411C950 D287E2D6BA2F8656D0869E03D988CEBF7B836D61       1   
FFD9607191459984619642FFDE109415FBD135D4 150A285643FD1A1F75B3DAB3B909C60402F03D76       1   
FFDFE2EDE8A46EF89F4E670028002763FC719E22 32BAC780CED052F95E2A8A8536F48264A1C08756       1   
FFE754BB2D984795616DAF79F744A209E4770B12 28252AC948BB1B4419C2CB07D6DDBAF763F341BF       1   

                                                                                   F3_kWh  \
user_code                                customer_code                                      
00106F4E6FEEFB8C81BADE0E7E985D85DFA3B9D9 B65DF5CEC830DF4EA6D402004F67F8597DC277DB       1   
                                         D1C6F205FC9B53080CD029948223269294E37770       1   
0045FE85440E191D5FEE535579C0FAA07A92D051 C82AB2965E72B4AF88C31C0D49A

In [9]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

908K	/data/invoices_sample.csv


In [10]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [11]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 187.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42607,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 187.40 GiB
Comm: tcp://127.0.0.1:37389,Total threads: 1
Dashboard: http://127.0.0.1:44497/status,Memory: 187.40 GiB
Nanny: tcp://127.0.0.1:39905,


In [272]:
df.dtypes

bill_id                            int64
F1_kWh                           float64
F2_kWh                           float64
F3_kWh                           float64
city                              object
address                           object
nominative                        object
sex                               object
age                                int64
user_code                         object
date                       datetime64[s]
light_start_date           datetime64[s]
light_end_date             datetime64[s]
tv                                object
gas_amount                        object
gas_average_cost                  object
light_average_cost               float64
emission_date              datetime64[s]
supply_type                       object
gas_start_date             datetime64[s]
gas_end_date               datetime64[s]
extra_fees                        object
gas_consumption                  float64
light_consumption                float64
gas_offer       

In [105]:
import re
#df['light_amount'].str.replace(',', '.').str.extract('-?([0-9]*\.[0-9]+)') 
print(cudf.to_numeric(df['light_amount'].str.replace(',', '.'), errors='coerce').isna().sum()  )
df['light_amount'].isna().sum()

752


744

In [130]:
df['average_gas_bill_cost'].str.replace(',', '.').str.extract('-?([0-9]*\.[0-9]+)').astype(float)

,0,1
0,<NA>,<NA>
1,<NA>,<NA>
2,<NA>,<NA>
3,<NA>,<NA>
4,<NA>,<NA>
...,...,...
1995,<NA>,<NA>
1996,<NA>,<NA>
1997,<NA>,<NA>
1998,<NA>,<NA>


In [144]:
df['average_gas_bill_cost'].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]

'€/smc'

In [177]:
def col_type(df, find=['numeric']):
    """
    Return a list containing all columns of the specified type
    df : dataframe
    type : list or single value with the specified value/s
    """
    tipo = []
    if type(find) == list:
        tipo.extend(find)
    else:
        tipo.append(find)
    
    col = set()
    
    for i in tipo:
        if i == 'numeric':
            col.update(df.select_dtypes(include=np.number).columns)
        elif i == 'float':
            col.update(df.select_dtypes(include=float).columns)
        elif i == 'int':
            col.update(df.select_dtypes(include=int).columns)
        elif i == 'date':
            col.update(df.select_dtypes(include=np.datetime64).columns)
        elif i == 'string':
            col.update(df.select_dtypes(include=object).columns)
            
    return list(col)

In [179]:
col_type(df, find=['string', 'date'])

['average_gas_bill_cost',
 'light_start_date',
 'light_offer_type',
 'gas_average_cost',
 'gas_amount',
 'gas_start_date',
 'average_unit_light_cost',
 'supply_type',
 'extra_fees',
 'user_code',
 'howmuch_pay',
 'tv',
 'light_end_date',
 'average_unit_gas_cost',
 'address',
 'gas_end_date',
 'sex',
 'light_amount',
 'billing_frequency',
 'city',
 'average_light_bill_cost',
 'nominative',
 'total_amount',
 'emission_date',
 'customer_code',
 'date']

In [12]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(ddf.npartitions)
type(ddf)

5
CPU times: user 30.6 ms, sys: 17.6 ms, total: 48.1 ms
Wall time: 41.9 ms


dask_cudf.core.DataFrame

In [150]:
df['bill_type'].value_counts()

False    2000
Name: bill_type, dtype: int32

In [14]:
ddf.head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>


In [15]:
df = ddf.loc['0':'4']
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,object,object,object,object,int64,object,object,object,object,object,object,object,float64,object,object,object,object,object,float64,float64,uint64,object,uint64,object,object,object,object,object,object,object,object,int8,bool,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [16]:
df['date']

<dask_cudf.Series | 15 tasks | 5 npartitions>

In [ ]:
# LOCATE MISSING VALUES
rows = df.shape[0]
#wait(ddf.shape[0])
print("Total rows", rows)

# These are the rows with missing values
for c in columns:
    if (not base.locate_null_values(c).empty):
        print(c, "\t Affected rows:", base.locate_null_values(c).shape[0])
    #else:
        #print(c, "\t Qui no missing values")

Total rows 2000
address 		 Affected rows: 4
nominative 		 Affected rows: 4
gas_amount 		 Affected rows: 1120
gas_average_cost 		 Affected rows: 1496
light_average_cost 		 Affected rows: 1122
gas_consumption 		 Affected rows: 1120
light_consumption 		 Affected rows: 744
light_amount 		 Affected rows: 744
average_unit_light_cost 		 Affected rows: 1115
average_light_bill_cost 		 Affected rows: 1122
average_unit_gas_cost 		 Affected rows: 1496
average_gas_bill_cost 		 Affected rows: 1494
billing_frequency 		 Affected rows: 1569
gas_system_charges 		 Affected rows: 1155
light_system_charges 		 Affected rows: 776
gas_material_cost 		 Affected rows: 1155
light_transport_cost 		 Affected rows: 775
gas_transport_cost 		 Affected rows: 1155
light_material_cost 		 Affected rows: 775


In [18]:
#Visto che address e nominative hanno lo stesso numero di NA, vale la 
#pena controllare ed eventualmente eliminare le righe?

base.locate_null_values('nominative')

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
1611,1611,23311.0,2219.0,467.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,8CE314D06FEFFD54C0F1497C20C479F376102747,...,<NA>,F631ED0EF8EC71428366F15C369AA53792A6F20D,monthly,False,<NA>,1176.31,<NA>,481.81,<NA>,2067.89
1627,1627,210.0,149.0,226.0,VIETRI DI POTENZA,<NA>,<NA>,P,45,A1E7C992D46A2D6D911C3595DF603D3F2C6BEB37,...,<NA>,1E9BC1B19300B6E472980894714A4C8382C565EA,monthly,False,<NA>,76.08,<NA>,50.45,<NA>,85.57
1639,1639,95.0,52.0,105.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,3CE203296BE19C1257020DA7EFEB84A0D3F4F7B7,...,<NA>,CC5C0DB0AA0034008B3398775B1EC89FC21A9DDB,monthly,False,<NA>,40.80,<NA>,29.47,<NA>,45.65
1858,1858,65.0,34.0,60.0,VIETRI DI POTENZA,<NA>,<NA>,P,66,D3117D65827932EB6A7F54F73EB1700C0D9F289C,...,<NA>,EB24C1C62642D39E65FB5BA50044153D2BF6EEC9,monthly,False,<NA>,44.17,<NA>,36.34,<NA>,47.11


In [19]:
gas = df[['average_gas_bill_cost','gas_consumption','gas_amount','gas_average_cost','gas_transport_cost', 'gas_system_charges', 'gas_material_cost']]
gas

,average_gas_bill_cost,gas_consumption,gas_amount,gas_average_cost,gas_transport_cost,gas_system_charges,gas_material_cost
npartitions=5,,,,,,,
,object,float64,object,object,float64,float64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [172]:
#search by pattern
def search_by_pattern(pattern):
    import re, itertools
    pattern_list = df.columns.str.contains(re.compile(pattern)).tolist()
    #print(gas_list)
    #df.loc[:, itertools.compress(df.columns.tolist(), gas_list)].drop('gas_offer', axis=1)
    return list(itertools.compress(df.columns.tolist(), pattern_list))

In [173]:
date_col_list = search_by_pattern('date')

In [21]:
# Le date potrebbero fornirci informazioni sulla billing_frequency, o viceversa, come lo stabiliamo?
# Cosa indica la colonna dell'emission_date??

df.loc[0, 'billing_frequency']

<dask_cudf.Series | 15 tasks | 5 npartitions>

In [22]:
# LOCATE OUTLIERS

In [23]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf
import time

cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("150GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("100GB"),
)
client = Client(cluster)

time.sleep(2)
print("cluster status ", cluster.status)
print("cluster information ", cluster)
print("client information ",client)
client

/opt/conda/envs/rapids/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46451 instead
  f"Port {expected} is already in use.\n"


cluster status  Status.running
cluster information  LocalCUDACluster(3b1845c7, 'tcp://127.0.0.1:46009', workers=1, threads=1, memory=187.40 GiB)
client information  <Client: 'tcp://127.0.0.1:46009' processes=1 threads=1, memory=187.40 GiB>


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:46451/status,
Dashboard: http://127.0.0.1:46451/status,Workers: 1
Total threads: 1,Total memory: 187.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46009,Workers: 1
Dashboard: http://127.0.0.1:46451/status,Total threads: 1
Started: Just now,Total memory: 187.40 GiB
Comm: tcp://127.0.0.1:35273,Total threads: 1
Dashboard: http://127.0.0.1:37873/status,Memory: 187.40 GiB
Nanny: tcp://127.0.0.1:35707,


In [24]:
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="0.1 GiB")
print(ddf.npartitions)
ddf = ddf.quantile(q=0.5)
_ = wait(ddf)

48


In [25]:
!nvidia-smi

Wed Feb 16 15:16:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:65:00.0 Off |                    0 |
| N/A   81C    P0    36W /  70W |   8676MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
ddf.compute()

MemoryError: std::bad_alloc: CUDA error at: /opt/conda/envs/rapids/include/rmm/mr/device/cuda_memory_resource.hpp

In [ ]:
def column_outliers(x:cudf.Series, k:float):
    q1 = x.quantile(q=0.25)
    q3 = x.quantile(q=0.75)
    I = q3 - q1
    return (x < q1 - k*I) or (x > q3 + k*I)

def df_outliers(df:cudf.DataFrame, k:float):
    q1 = df.quantile(q=0.25)
    q3 = df.quantile(q=0.75)
    I = q3 - q1
    return (x < q1 - k*I) or (x > q3 + k*I)

In [ ]:
col_outliers(df.iloc[:,1], 0)

In [ ]:
type = ddf.dtypes
num = type[type!='object'][type!='bool']
num.index

In [ ]:
num_df = ddf[num.index].quantile(q=0.2)
num_df = num_df.persist()
num_df.compute()

In [ ]:
# SORT DATA
# Idee: ordinare per data

In [ ]:
''' 
+++++ DATA VALIDATION +++++
    • Check data range
    • Check column uniqueness
    • Find data-mismatched data types
'''

In [ ]:
# CHECK DATA RANGE

In [ ]:
# Controllo se chi ha P ha età nulla
df.loc[(df['sex']=='P') & (df['age']>=0), 'age'] = np.nan
df.loc[(df['sex']=='P') & (np.isnan(df['age']))]

In [ ]:
# Age >= 18
# Age non ha valori nulli (già controllato in precedenza IN TEORIA)
#
df['age'].isna().sum()

In [147]:
# Che fare con i minorenni? Li mettiamo tutti a 18 anni?
df[df['age'] < 18]

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
5,5,0.0,0.0,0.0,VALGRANA,CF9E4E3F659E5A7A5C9975570E66C5985FF20486,B2D49CC3EA712D9A6F333F57EB9280601924EC37,P,0,D3BFE563AB14EA63F311B8A25B3AA8F786718CCD,...,"0,86 €/smc",9ACA67CB28293E711CCA5F9B4FB0B0373FB5B36C,<NA>,False,16.97,<NA>,167.54,<NA>,44.45,<NA>
23,23,0.0,0.0,0.0,PONTERANICA,A5BECF7E67B192B00C4B68BA490479C036BE324E,AACEECF6549DC6B1D7E74EE72C93C53B7FAA7821,P,0,9CE05516185F3276F00525216AB198FE14B5113F,...,<NA>,95F77EB44034D838442434D344852734EA5BDAC0,<NA>,False,6.75,<NA>,-19.23,<NA>,-122.49,<NA>
71,71,6513.0,5302.0,6610.0,SAN MARTINO IN PENSILIS,8A8AC87E60AABF6E2BC6C4425707B36DC9342769,E5E3636009814DCE7967BD47310860DB493AA915,P,0,9D25EFBF38408B492D5C613EA3113B87285872EA,...,<NA>,B2E4410FBDBDB81768C594A8D81CA38462ABC32F,<NA>,False,<NA>,750.58,<NA>,190.01,<NA>,1100.63
95,95,0.0,0.0,0.0,ROVITO,87910E31B8BA24C489B9688D5EB84A75598078D9,C1DE0590BE87ED55771594FC3F74060343F7D53E,P,0,BBA1A913CD9C5AEAD3C8437CF898917989D8EE92,...,<NA>,82533FF2E816CD46653F94D7C1CC7E3EDD1B7A81,<NA>,False,<NA>,-190.71,<NA>,-114.67,<NA>,-200.04
99,99,53.0,2.0,3.0,BOTRUGNO,E75DD5B4053299620255129FCA769710E94F4D80,A14C1A3CACE858F3C127E41C86A5A2FBC92CD6AA,P,0,251E78818EFF19CEC91BD58A323407705F8B03A2,...,<NA>,F4004E858536929AE313C2702F09B91593D7838D,<NA>,False,<NA>,8.42,<NA>,5.84,<NA>,13.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,1972,42.0,6.0,0.0,PISANO,8B39CE1D2F14567B1648B7D4D408C69E171D1425,6BC3649005162376FDC8643244B768EA39D7671F,P,0,D4890BAB1F93EE96370D5B71A194587A25912405,...,<NA>,9E91301945B8EDB91F7DC26D6B433CB8C628D1C1,monthly,False,<NA>,11.72,<NA>,9.42,<NA>,20.34
1974,1974,75.0,59.0,103.0,BOTRUGNO,6ACE234AF417B73B6B0A9F8E9DE0349FB4BED941,DE7E423B3CD8E06D1ED6D379547B6BE1482D0AC8,P,0,FF4B3953260A64357143F82FD2881C2FDE653DE5,...,<NA>,F9880177E7100F50F3A24B5E0727FEFE311AD3EA,monthly,False,<NA>,40.63,<NA>,29.88,<NA>,35.03
1980,1980,128.0,70.0,113.0,MALVICINO,DC05160B563E3CD82EF4F2A2D45F82E9B48154C7,091E23C5253C4DEB06F086D8604499615B413C6C,P,0,40BF5FE9E0EF59A68C0BA0DF41EA2348066384F2,...,<NA>,DB010BAE2C7A146433B38CF023FC6DF54CEF4EAF,monthly,False,<NA>,24.81,<NA>,11.77,<NA>,39.51
1991,1991,210.0,37.0,50.0,ERICE,0477933B221696C5D103CD0894BF4D041ACBFCE6,1E11DD53B7F53999CB83D82891930B9B1C43973D,P,0,4CBC343278DC588F312015BC69CFE9B1C89901F2,...,<NA>,1F9DB5F29F888E7D1B16A2D59E8F5EC0609E1FBC,monthly,False,<NA>,32.91,<NA>,20.12,<NA>,46.63


In [188]:
# Dobbiamo metterli in inglese? Si può fare con un map
ing = {
    'luce': 'light',
    'gas e luce': 'gas and light',
    'gas' : 'gas',
}
df['supply_type'] = df['supply_type'].map(ing)

In [191]:
df['supply_type'].value_counts(dropna=False)

light            1116
gas               744
gas and light     140
Name: supply_type, dtype: int32

In [192]:
# Che fare qua? Billing frequency (‘monthly’, ‘quarterly’, etc.)
df['billing_frequency'].value_counts(dropna=False)

<NA>        1569
monthly      422
bimester       9
Name: billing_frequency, dtype: int32

In [193]:
# Kind of invoice (False means a “standard bill”)
df['bill_type'].value_counts(dropna=False)

False    2000
Name: bill_type, dtype: int32

In [194]:
# Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
df['light_offer_type'].value_counts(dropna=False)

light single zone    1079
light                 744
light multi zones     103
ligth bizone           43
light                  31
Name: light_offer_type, dtype: int32

In [ ]:
# CHECK COLUMN UNIQUENESS

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

# ATTENZIONE!! Probabile che le date del gas siano invertite, trovare un modo per confermarlo

In [260]:
# Dovrebbe accorgersi di colonne con similarità > soglia
def get_duplicate_col(df, soglia=0.9):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """  
        cols = df.columns.values
        #return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
        
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if len(set(cols[i]) & set(cols[j])) / float(len(set(cols[i]) | set(cols[j]))) > soglia ]

In [263]:
def get_duplicate_col_V2(df, soglia=0.9):
    cols = df.columns.values
    duplicate_col = []
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            if col[i].shape[0] - col[i].compare(col[j]).shape[0] / float(col[i].shape[0]) > soglia:
                duplicate_col.append(col[i], col[j])
                
    return duplicate_col

In [270]:
#provo a concatenare lo stesso dataset più volte solo per vedere come regge lo scaling di dati
big_df = cudf.concat([df for _ in range(100)], sort=False) #200k con la paglia

get_duplicate_col(big_df, soglia=0.9)

[('light_average_cost', 'average_light_bill_cost'),
 ('average_light_bill_cost', 'average_gas_bill_cost'),
 ('gas_material_cost', 'light_material_cost')]

In [126]:
base.get_duplicate_columns()
# Ne vengono individuate solo 5 di coppie 100% fedeli, 
# ma cambia la questione se abbassiamo la soglia? Ci sono degli errori sopra perchè 
# quelle 100% uguali non vengono riconosciute, mentre qua sì

[('date', 'light_start_date'),
 ('date', 'gas_end_date'),
 ('light_start_date', 'gas_end_date'),
 ('light_end_date', 'gas_start_date'),
 ('gas_average_cost', 'average_unit_gas_cost')]

In [ ]:
#FUFFA

compariser = cudf.Series(np.random.randn(2000))
print(df['gas_average_cost'])
print(df['average_unit_gas_cost'])
print(compariser.shape)
compariser = (df['gas_average_cost'] == df['average_unit_gas_cost']).astype(int)
print(compariser.sum())

In [ ]:
# FIND DATA MISMATCH

In [169]:
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}

In [170]:
def convert_to_datetime(col): 
    if col.dtype == object:
        a = cudf.DataFrame()
        a[['day', 'month', 'year']] = col.str.split(expand=True)
        a['month'] = a.month.str.lower().map(mesi)
        col = cudf.to_datetime(a)
    return col

In [174]:
for col in date_col_list:
    df[col] = convert_to_datetime(df[col])
df[date_col_list]

,date,light_start_date,light_end_date,emission_date,gas_start_date,gas_end_date
0,2018-04-16,2018-04-16,2019-11-25,2020-12-31,2019-11-25,2018-04-16
1,2020-12-05,2020-12-05,2020-12-31,2020-12-31,2020-12-31,2020-12-05
2,2020-12-05,2020-12-05,2020-12-31,2020-12-31,2020-12-31,2020-12-05
3,2020-10-03,2020-10-03,2020-12-31,2020-12-31,2020-12-31,2020-10-03
4,2020-12-16,2020-12-16,2020-12-31,2020-12-31,2020-12-31,2020-12-16
...,...,...,...,...,...,...
1995,2020-12-05,2020-12-05,2021-01-12,2021-01-12,2021-01-12,2020-12-05
1996,2020-12-05,2020-12-05,2021-01-12,2021-01-12,2021-01-12,2020-12-05
1997,2020-12-05,2020-12-05,2021-01-12,2021-01-12,2021-01-12,2020-12-05
1998,2020-12-05,2020-12-05,2021-01-12,2021-01-12,2021-01-12,2020-12-05


In [145]:
df['date'] == df['light_start_date']

0       True
1       True
2       True
3       True
4       True
        ... 
1995    True
1996    True
1997    True
1998    True
1999    True
Length: 2000, dtype: bool

In [180]:
# Seguendo il GroundTruth mystico
mappone = {'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'str',
 'address': 'str',
 'nominative': 'str',
 'sex': 'str',
 'age': 'int64',
 'user_code': 'str',
 'date': 'datetime64[s]',
 'light_start_date': 'datetime64[s]',
 'light_end_date': 'datetime64[s]',
 'tv': 'float64',
 'gas_amount': 'float64',
 'gas_average_cost': 'float64',
 'light_average_cost': 'float64',
 'emission_date': 'datetime64[s]',
 'supply_type': 'str',
 'gas_start_date': 'datetime64[s]',
 'gas_end_date': 'datetime64[s]',
 'extra_fees': 'float64',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'float64',
 'light_offer_type': 'str',
 'light_offer': 'str',
 'howmuch_pay': 'float64',
 'total_amount': 'float64',
 'light_amount': 'float64',
 'average_unit_light_cost': 'float64',
 'average_light_bill_cost': 'float64',
 'average_unit_gas_cost': 'float64',
 'average_gas_bill_cost': 'float64',
 'customer_code': 'str',
 'billing_frequency': 'str',
 'bill_type': 'str',
 'gas_system_charges': 'float64',
 'light_system_charges': 'float64',
 'gas_material_cost': 'float64',
 'light_transport_cost': 'float64',
 'gas_transport_cost': 'float64',
 'light_material_cost': 'float64'
}

In [183]:
#Check sui tipi di dato attuali
type_dict = base.get_columns_types()
type_dict

{'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'object',
 'address': 'object',
 'nominative': 'object',
 'sex': 'object',
 'age': 'int64',
 'user_code': 'object',
 'date': 'datetime64[s]',
 'light_start_date': 'datetime64[s]',
 'light_end_date': 'datetime64[s]',
 'tv': 'object',
 'gas_amount': 'object',
 'gas_average_cost': 'object',
 'light_average_cost': 'float64',
 'emission_date': 'datetime64[s]',
 'supply_type': 'object',
 'gas_start_date': 'datetime64[s]',
 'gas_end_date': 'datetime64[s]',
 'extra_fees': 'object',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'uint64',
 'light_offer_type': 'object',
 'light_offer': 'uint64',
 'howmuch_pay': 'object',
 'total_amount': 'object',
 'light_amount': 'object',
 'average_unit_light_cost': 'object',
 'average_light_bill_cost': 'object',
 'average_unit_gas_cost': 'object',
 'average_gas_bill_cost': 'object',
 'customer_code': 'object',
 'billing_frequency': 'o

In [182]:
# setto le colonne con i tipi di dato in mappone
df = base.cast_columns_types(mappone)

ValueError: Could not convert strings to float type due to presence of non-floating values.

In [184]:
# in 'bill_type' False == 'standard bill'
df = base.replace('bill_type', 'False', 'standard bill', False)
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,26.0,25.0,33.0,CINTANO,2D244E5D90065985546AEA4D52F54C9705B8038B,102CADC9FC8C745FE3B8C27F7267E07C9691728A,M,55,6D3FDD0A2EEEE1A2C1AC23E6C316048DD2CCE859,...,<NA>,53D29ACDEB11F45D4F305A4D00E5F03C1F2CB674,monthly,False,<NA>,13.08,<NA>,7.62,<NA>,15.65
1996,1996,71.0,61.0,88.0,TREZZONE,A01243282C24F561D6B54F1C1FCB45AE80F75B0A,9C3B5B4093FC99569E5BBA918A977681A46F52DF,M,48,52315237533E1A6504E3795EAC1D5586744C62FD,...,<NA>,2B8D02A2AD117EC401BCEF3C0651FEEBB6609D1C,monthly,False,<NA>,9.2,<NA>,8.76,<NA>,25.8
1997,1997,37.0,71.0,100.0,RIPATRANSONE,469AB252FF0A2C60B32D0337C933E3565E88C0C2,1B451B6AA312EFF93372D44EC6C24C0110A628A1,F,32,2777855F19667FBFEDEAEF181E7ABC8584262127,...,<NA>,A1BFA0E4833E27AB398F4420414E037D250F0A7E,monthly,False,<NA>,8.7,<NA>,8.66,<NA>,24.14
1998,1998,74.0,78.0,92.0,MONSELICE,9E24BB26322BD95D11E26DFB80AA1168C9661FDC,F6BADD2456E283A8DCA6F3FF640724072108BAEF,F,54,8DC8820FB50005ED9CC4E7C187B9703460941856,...,<NA>,DED078288677E4E8A15874405E79C26AC78747B0,monthly,False,<NA>,10.21,<NA>,8.96,<NA>,30.43


In [ ]:
df['bill_type'].value_counts()

In [109]:
# Check degli altri data type sulla base delle regole date
# Possibile SOLO MANUALE? O Esiste un'alternativa automatica?
df.dtypes.sort_values()

bill_type                           bool
bill_id                            int64
light_offer                       uint64
gas_offer                         uint64
age                                int64
light_transport_cost             float64
gas_material_cost                float64
light_system_charges             float64
gas_system_charges               float64
light_consumption                float64
gas_consumption                  float64
gas_transport_cost               float64
gas_end_date               datetime64[s]
gas_start_date             datetime64[s]
emission_date              datetime64[s]
light_average_cost               float64
light_material_cost              float64
F1_kWh                           float64
F2_kWh                           float64
light_end_date             datetime64[s]
light_start_date           datetime64[s]
date                       datetime64[s]
F3_kWh                           float64
gas_amount                        object
city            

In [204]:
df['gas_offer'].dtype

dtype('uint64')

In [205]:
# MEGA MAP WAITING ROOM

In [ ]:
data_types {
    'user_code': string,
    ...
}

In [170]:
# Non c'è nessuna riga tutta nulla, Giulio è contento
df.dropna(how='all')

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,26.0,25.0,33.0,CINTANO,2D244E5D90065985546AEA4D52F54C9705B8038B,102CADC9FC8C745FE3B8C27F7267E07C9691728A,M,55,6D3FDD0A2EEEE1A2C1AC23E6C316048DD2CCE859,...,<NA>,53D29ACDEB11F45D4F305A4D00E5F03C1F2CB674,monthly,False,<NA>,13.08,<NA>,7.62,<NA>,15.65
1996,1996,71.0,61.0,88.0,TREZZONE,A01243282C24F561D6B54F1C1FCB45AE80F75B0A,9C3B5B4093FC99569E5BBA918A977681A46F52DF,M,48,52315237533E1A6504E3795EAC1D5586744C62FD,...,<NA>,2B8D02A2AD117EC401BCEF3C0651FEEBB6609D1C,monthly,False,<NA>,9.2,<NA>,8.76,<NA>,25.8
1997,1997,37.0,71.0,100.0,RIPATRANSONE,469AB252FF0A2C60B32D0337C933E3565E88C0C2,1B451B6AA312EFF93372D44EC6C24C0110A628A1,F,32,2777855F19667FBFEDEAEF181E7ABC8584262127,...,<NA>,A1BFA0E4833E27AB398F4420414E037D250F0A7E,monthly,False,<NA>,8.7,<NA>,8.66,<NA>,24.14
1998,1998,74.0,78.0,92.0,MONSELICE,9E24BB26322BD95D11E26DFB80AA1168C9661FDC,F6BADD2456E283A8DCA6F3FF640724072108BAEF,F,54,8DC8820FB50005ED9CC4E7C187B9703460941856,...,<NA>,DED078288677E4E8A15874405E79C26AC78747B0,monthly,False,<NA>,10.21,<NA>,8.96,<NA>,30.43


In [ ]:
''' 
+++++ DATA STRUCTURING +++++
    • Change column data types
    • Delete, split or merge columns
    • Pivot and unpivot
'''

In [ ]:
# CHANGE COLUMN DATA TYPES 
# aggregabile con lo step precedente

In [ ]:
# DELETE, SPLIT OR MERGE COLUMNS
# rimozione colonne duplicate 

In [ ]:
# PIVOT AND UNPIVOT
# non saprei cosa farci

In [ ]:
''' 
+++++ DATA ENRICHMENT +++++
    • Calculate columns using expressions
    • Set primary key column
    • Join or append DataFrames
    • Group by and aggregate records
    • Scale column values into a certain range
    • Encode categorical data (one-hot encoding and label encoding)
'''

In [ ]:
''' 
+++++ DATA FILTERING +++++
    • Sample rows
    • Select a subset of rows through a user-defined query
'''

In [ ]:
# SAMPLE ROWS
df.sample(20)

In [ ]:
# SELECT A SUBSET OF ROWS?!?

In [ ]:
''' 
+++++ DATA CLEANING +++++
    • Change data format and case
    • Deduplicate data
    • Missing value imputation
    • Find and replace values
'''

In [ ]:
# CHANGE DATA FORMAT AND CASE
# data format dovrebbe già essere fatto a questo punto
# case pure se vogliamo aver fatto tutto per bene

In [ ]:
# DEDUPLICATE DATA
# controllo se ho più bollette uguali dello stesso cliente

In [ ]:
# MISSING VALUE IMPUTATION
# qua? Spero di averne pochissimi nel caso

In [ ]:
# FIND AND REPLACE VALUES
# ossia?